In [1]:
import numpy as np
import logging
import math
import pandas as pd
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
import os

In [3]:
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, GroupKFold

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC


In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import FactorAnalysis

In [7]:
output_path = "/home/ashish/Results/Datasets/Shimmer/MP/Catch22/{}/"
data_type = "default"
SEED_VALUES = [103007, 1899797, 191099]

In [14]:
list_clf = [
            RidgeClassifier(), 
            LogisticRegression(C=0.001, max_iter=150),
            RandomForestClassifier(),           
            GaussianNB(), 
            KNeighborsClassifier(n_neighbors=40),
            #ExtraTreesClassifier(n_estimators=100)
            svm.SVC(kernel="linear", C=0.025), 
            ]


name_clf = [
            "RidgeClassifier",
            "LogisticRegression",
            "RandomForestClassifier",       
            "GaussianNB",
            "KNeighborsClassifier",
            #ExtraTreesClassifier(n_estimators=100)
            "svm.SVC"
            ]

In [25]:
sv = 103007
output_path = "/home/ashish/Results/Datasets/Shimmer/MP/Catch22/{}/"
data_type = "default"


train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(os.path.join(output_path.format(sv), "y_train_{}.csv.npy".format(data_type)), allow_pickle=True)
test_Y = np.load(os.path.join(output_path.format(sv), "y_test_{}.npy".format(data_type)), allow_pickle=True)
    # train_df = shuffle(train_df)
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(1426, 990) (595, 990) (1426,) (595,)


In [6]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 600, num = 8)]
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               # 'bootstrap': bootstrap
              }

In [14]:
max_depth

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]

In [89]:
output_path = "/home/ashish/Results/Datasets/Shimmer/MP/Catch22/{}/"
data_type = "default"
sv = 103007


train_pid = np.load(output_path.format(sv) + "TRAIN_{}_pid.npy".format(data_type), allow_pickle=True)

temp_pid = train_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]
groups = {"g1":[], "g2":[], "g3":[], "g4":[]}
unique_pid = list(np.unique(temp_pid))
groups["g1"].extend(unique_pid[0:9])
groups["g2"].extend(unique_pid[9:18])
groups["g3"].extend(unique_pid[18:27])
groups["g4"].extend(unique_pid[27:])


groups_list = []

for i in temp_pid:
    if i in groups["g1"]:
        groups_list.append(1)
    elif i in groups["g2"]:
        groups_list.append(2)
    elif i in groups["g3"]:
        groups_list.append(3)
    else:
        groups_list.append(4)
        
        

# for i, model in enumerate(list_clf):
# print("--------------------------- {}----------------".format(name_clf[i]))
avg_acc = []
# for sv in SEED_VALUES:
train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(os.path.join(output_path.format(sv), "y_train_{}.csv.npy".format(data_type)), allow_pickle=True)
test_Y = np.load(os.path.join(output_path.format(sv), "y_test_{}.npy".format(data_type)), allow_pickle=True)
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)

#  ExtraTreesClassifier(n_estimators=50) LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)
model_features = SelectFromModel(lsvc, prefit=True)
train_X = model_features.transform(train_X)
test_X = model_features.transform(test_X)

# pca = PCA(n_components=700).fit(train_X)
# pca = TruncatedSVD(n_components=800, n_iter=7, random_state=42).fit(train_X)
# VarianceThreshold(threshold=(.6 * (1 - .7))).fit(train_X)
# 
# FactorAnalysis(n_components=600, random_state=0).fit(train_X)
# pca = SelectKBest(mutual_info_classif, k=600).fit(train_X, train_Y)
# train_X = pca.transform(train_X)
# test_X = pca.transform(test_X)

# sc = StandardScaler()
# train_X = sc.fit_transform(train_X)
# test_X = sc.transform(test_X)

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

# model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, 
#                                random_state=42, n_jobs = -1)

# lm = [100000, 10000, 1000, 1, 0.01, 0.001, 0.0001, 0.00001]

group_kfold = GroupKFold(n_splits=4)
# for p in lm:
# model =  LogisticRegression(C=0.01, max_iter=400) 
model =  RidgeClassifier(alpha=300) 

scores = cross_val_score(model, train_X, train_Y, cv=group_kfold, groups=groups_list, scoring="accuracy")
print("--------------{}-----------".format(p))
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# model.fit(train_X, train_Y)
# pred_Y = model.predict(test_X)
# accuracy = metrics.accuracy_score(test_Y, pred_Y)
# print(accuracy)
    # model.fit(train_X, train_Y)
# pred_Y = model.predict(test_X)

# regressor = RandomForestClassifier(n_estimators=200, max_depth=50)
# model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
#                            random_state=42, n_jobs = -1)
# svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 
# model =   RidgeClassifier(alpha=1000) #LogisticRegression(C=0.001, max_iter=150)
# model.fit(train_X, train_Y)
# pred_Y = model.predict(test_X)


# accuracy = metrics.accuracy_score(test_Y, pred_Y)
# confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
# classification_report = metrics.classification_report(test_Y, pred_Y)

# print(accuracy) #, print(confusion_matrix), print(classification_report)
# avg_acc.append(accuracy)

# print(round(np.mean(avg_acc),2), np.std(avg_acc))


# LR 0.01

(1426, 990) (595, 990) (1426,) (595,)
(1426, 263) (595, 263) (1426,) (595,)
--------------1e-05-----------
0.85 accuracy with a standard deviation of 0.04


0.76, 


0.77,  LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y) is the best setting

300 semms best alpha for RidgeRegression rowing

400 for mp

In [99]:
output_path = "/home/ashish/Results/Datasets/Shimmer/Rowing/Catch22/{}/"
data_type = "default"


# for i, model in enumerate(list_clf):
# print("--------------------------- {}----------------".format(name_clf[i]))
avg_acc = []
for sv in SEED_VALUES:
    train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
    test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))
    
    train_Y = np.load(os.path.join(output_path.format(sv), "y_train_{}.csv.npy".format(data_type)), allow_pickle=True)
    test_Y = np.load(os.path.join(output_path.format(sv), "y_test_{}.npy".format(data_type)), allow_pickle=True)
    print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)
    

    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)
    
    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)
    model_features = SelectFromModel(lsvc, prefit=True)
    train_X = model_features.transform(train_X)
    test_X = model_features.transform(test_X)
    
    print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

    # regressor = RandomForestClassifier()
    # model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
    #                            random_state=42, n_jobs = -1)
    # svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 
    # model = LogisticRegression(C=1, max_iter=500)#LogisticRegression(C=0.001, max_iter=150)
    model =  RidgeClassifier(alpha=300)  #RidgeClassifier(alpha=400) 

    model.fit(train_X, train_Y)
    pred_Y = model.predict(test_X)


    accuracy = metrics.accuracy_score(test_Y, pred_Y)
    confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
    classification_report = metrics.classification_report(test_Y, pred_Y)

    print(accuracy) #, print(confusion_matrix), print(classification_report)
    avg_acc.append(accuracy)

print(round(np.mean(avg_acc),2), np.std(avg_acc))

(1773, 990) (760, 990) (1773,) (760,)
(1773, 346) (760, 346) (1773,) (760,)
0.7657894736842106
(1803, 990) (730, 990) (1803,) (730,)
(1803, 333) (730, 333) (1803,) (730,)
0.7082191780821918
(1773, 990) (760, 990) (1773,) (760,)
(1773, 347) (760, 347) (1773,) (760,)
0.7513157894736842
0.74 0.024452094624567374


# MP 990

- LogisticRegression 0.84 0.015
- RandomForestClassifier 0.81 0.02
- svm.SVC(kernel='linear', C=0.001) 0.82 0.01
- RidgeClassifier(alpha=1000) 0.83 0.031


# Rowing 990

- LogisticRegression 0.75 0.015
- RandomForestClassifier 0.72 0.025
- svm.SVC(kernel='linear', C=0.001) 0.74 0.01
- RidgeClassifier(alpha=1000) 0.74 0.0244

In [7]:
output_path = "/home/ashish/Results/Datasets/Shimmer/MP/tsfresh/{}/"
data_type = "default"
sv = 103007

train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(output_path.format(sv) + "y_train_{}.npy".format(data_type), allow_pickle=True)
test_Y = np.load(output_path.format(sv) + "y_test_{}.npy".format(data_type), allow_pickle=True)

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(1426, 15893) (595, 15893) (1426,) (595,)


In [114]:
output_path = "/home/ashish/Results/Datasets/Shimmer/MP/tsfresh/{}/"
data_type = "default"


sv = 103007
train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(output_path.format(sv) + "y_train_{}.npy".format(data_type), allow_pickle=True)
test_Y = np.load(output_path.format(sv) + "y_test_{}.npy".format(data_type), allow_pickle=True)

train_pid = np.load(output_path.format(sv) + "TRAIN_{}_pid.npy".format(data_type), allow_pickle=True)
test_pid = np.load(output_path.format(sv) + "TEST_{}_pid.npy".format(data_type), allow_pickle=True)

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


temp_pid = train_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]
groups = {"g1":[], "g2":[], "g3":[], "g4":[]}
unique_pid = list(np.unique(temp_pid))
groups["g1"].extend(unique_pid[0:9])
groups["g2"].extend(unique_pid[9:18])
groups["g3"].extend(unique_pid[18:27])
groups["g4"].extend(unique_pid[27:])


groups_list = []

for i in temp_pid:
    if i in groups["g1"]:
        groups_list.append(1)
    elif i in groups["g2"]:
        groups_list.append(2)
    elif i in groups["g3"]:
        groups_list.append(3)
    else:
        groups_list.append(4)

        
    
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)


#  ExtraTreesClassifier(n_estimators=50) LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)
model_features = SelectFromModel(lsvc, prefit=True)
train_X = model_features.transform(train_X)
test_X = model_features.transform(test_X)

# pca = PCA(n_components=1000).fit(train_X)
# pca = TruncatedSVD(n_components=800, n_iter=7, random_state=42).fit(train_X)
# VarianceThreshold(threshold=(.6 * (1 - .7))).fit(train_X)
# 
# FactorAnalysis(n_components=600, random_state=0).fit(train_X)
# pca = SelectKBest(mutual_info_classif, k=600).fit(train_X, train_Y)
# train_X = pca.transform(train_X)
# test_X = pca.transform(test_X)

lm = [100000, 10000, 1000, 1, 0.01, 0.001, 0.0001, 0.00001]
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


# regressor = RandomForestClassifier()
# model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
#                            random_state=42, n_jobs = -1)
# svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 

group_kfold = GroupKFold(n_splits=4)

# for l in lm:
model =  RidgeClassifier(alpha=400) # LogisticRegression(C=0.01, max_iter=500,) #LogisticRegression(C=0.001, max_iter=150)
scores = cross_val_score(model, train_X, train_Y, cv=group_kfold, groups=groups_list, scoring="accuracy")
# print("--------------{}-----------".format(l))
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


# model.fit(train_X, train_Y)
# pred_Y = model.predict(test_X)


# accuracy = metrics.accuracy_score(test_Y, pred_Y)
# confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
# classification_report = metrics.classification_report(test_Y, pred_Y)

# print(accuracy) #, print(confusion_matrix), print(classification_report)


# mp 1500 in linearsvc
# rowing default
# 0.01 for both


(1426, 15893) (595, 15893) (1426,) (595,)
(1426, 540) (595, 540) (1426,) (595,)
0.94 accuracy with a standard deviation of 0.02


/home/ashish/VirtualEnvironments/mlv1/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [58]:
output_path = "/home/ashish/Results/Datasets/Shimmer/Rowing/tsfresh/{}/"
data_type = "default"


sv = 103007
train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(output_path.format(sv) + "y_train_{}.npy".format(data_type), allow_pickle=True)
test_Y = np.load(output_path.format(sv) + "y_test_{}.npy".format(data_type), allow_pickle=True)

train_pid = np.load(output_path.format(sv) + "TRAIN_{}_pid.npy".format(data_type), allow_pickle=True)

test_pid = np.load(output_path.format(sv) + "TEST_{}_pid.npy".format(data_type), allow_pickle=True)

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)

lm = [100000, 10000, 1000, 1, 0.01, 0.001, 0.0001, 0.00001]


# regressor = RandomForestClassifier()
# model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
#                            random_state=42, n_jobs = -1)
# svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 

group_kfold = GroupKFold(n_splits=4)

# for l in lm:
#     model =  LogisticRegression(C=l, max_iter=500,) #LogisticRegression(C=0.001, max_iter=150)
#     scores = cross_val_score(model, train_X, train_Y, cv=4, groups=groups_list, scoring="accuracy")
#     print("--------------{}-----------".format(l))
#     print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

for l in lm:
    scores = []
    model =  LogisticRegression(C=l, max_iter=500,)
    for train_index, test_index in group_kfold.split(train_X, train_Y, groups_list):
        X_train, X_test = train_X[train_index], train_X[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]

        model.fit(X_train, y_train)
        pred_Y = model.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, pred_Y)
        scores.append(accuracy)
    print("--------------{}-----------".format(l))
    print("%0.2f accuracy with a standard deviation of %0.2f" % (np.mean(scores), np.std(scores)))

# model.fit(train_X, train_Y)
# pred_Y = model.predict(test_X)


# accuracy = metrics.accuracy_score(test_Y, pred_Y)
# confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
# classification_report = metrics.classification_report(test_Y, pred_Y)

# print(accuracy) #, print(confusion_matrix), print(classification_report)



(1773, 16331) (760, 16331) (1773,) (760,)
--------------100000-----------
0.75 accuracy with a standard deviation of 0.07
--------------10000-----------
0.75 accuracy with a standard deviation of 0.07
--------------1000-----------
0.75 accuracy with a standard deviation of 0.07
--------------1-----------
0.77 accuracy with a standard deviation of 0.04
--------------0.01-----------
0.76 accuracy with a standard deviation of 0.04
--------------0.001-----------
0.76 accuracy with a standard deviation of 0.04
--------------0.0001-----------
0.74 accuracy with a standard deviation of 0.04
--------------1e-05-----------
0.67 accuracy with a standard deviation of 0.04


In [56]:
scores

[0.8283433133732535,
 0.7060185185185185,
 0.7571428571428571,
 0.7761904761904762]

In [51]:
len(groups_list)

1773

In [32]:
temp_pid = train_pid[:, 0]

In [34]:
temp_pid = [i.split("_")[0] for i in temp_pid]

In [45]:
groups = {"g1":[], "g2":[], "g3":[], "g4":[]}

In [46]:
temp_pid = train_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]
groups = {"g1":[], "g2":[], "g3":[], "g4":[]}
unique_pid = list(np.unique(temp_pid))
groups["g1"].extend(unique_pid[0:9])
groups["g2"].extend(unique_pid[9:18])
groups["g3"].extend(unique_pid[18:27])
groups["g4"].extend(unique_pid[27:])


groups_list = []

for i in temp_pid:
    if i in groups["g1"]:
        groups_list.append(1)
    elif i in groups["g2"]:
        groups_list.append(2)
    elif i in groups["g3"]:
        groups_list.append(3)
    else:
        groups_list.append(4)

In [47]:
groups

['P10', 'P11', 'P12', 'P13', 'P15', 'P16', 'P17', 'P18', 'P2']

In [48]:
groups_list = []

for i in temp_pid:
    if i in groups["g1"]:
        groups_list.append(1)
    elif i in groups["g2"]:
        groups_list.append(2)
    elif i in groups["g3"]:
        groups_list.append(3)
    else:
        groups_list.append(4)

In [145]:
output_path = "/home/ashish/Results/Datasets/Shimmer/Rowing/tsfresh/{}/"
data_type = "default"


# for i, model in enumerate(list_clf):
# print("--------------------------- {}----------------".format(name_clf[i]))
avg_acc = []
for sv in SEED_VALUES:
    train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
    test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

    train_Y = np.load(output_path.format(sv) + "y_train_{}.npy".format(data_type), allow_pickle=True)
    test_Y = np.load(output_path.format(sv) + "y_test_{}.npy".format(data_type), allow_pickle=True)
    print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)
    

    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)

    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False,  max_iter=1500).fit(train_X, train_Y)
    model_features = SelectFromModel(lsvc, prefit=True)
    train_X = model_features.transform(train_X)
    test_X = model_features.transform(test_X)
    print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


    # regressor = RandomForestClassifier()
    # model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
    #                            random_state=42, n_jobs = -1)
    # svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 
    model = LogisticRegression(C=0.01, max_iter=500) #LogisticRegression(C=0.01, max_iter=500)#LogisticRegression(C=0.001, max_iter=150)
    model.fit(train_X, train_Y)
    pred_Y = model.predict(test_X)


    accuracy = metrics.accuracy_score(test_Y, pred_Y)
    confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
    classification_report = metrics.classification_report(test_Y, pred_Y)

    print(accuracy) #, print(confusion_matrix), print(classification_report)
    avg_acc.append(accuracy)

print(round(np.mean(avg_acc),2), np.std(avg_acc))

(1773, 16331) (760, 16331) (1773,) (760,)
(1773, 809) (760, 809) (1773,) (760,)
0.8144736842105263
(1803, 16331) (730, 16331) (1803,) (730,)
(1803, 749) (730, 749) (1803,) (730,)
0.7780821917808219
(1773, 16331) (760, 16331) (1773,) (760,)
(1773, 785) (760, 785) (1773,) (760,)
0.8197368421052632
0.8 0.018520713466991606


# MP 15893 features

- LogisticRegression 0.88 0.02
- RandomForestClassifier 0.83 0.016
- svm.SVC(kernel='linear', C=0.001) 0.89 0.027
- RidgeClassifier(alpha=300) 0.87 0.015


# Rowing 16331

- LogisticRegression 0.8 0.01
- RandomForestClassifier 0.73 0.02
- svm.SVC(kernel='linear', C=0.001) 0.77 0.014
- RidgeClassifier(alpha=300) 0.79 0.024

In [9]:
def generate_cross_val_splits(pid, full_df):
    second_last_col = full_df.columns[-2]
    pid_excl_df = full_df[full_df[second_last_col]!=pid]
    pid_incl_df = full_df[full_df[second_last_col]==pid]
        
    train_X, train_Y = pid_excl_df.iloc[:, :-2], pid_excl_df.iloc[:, -1]
    test_X, test_Y = pid_incl_df.iloc[:, :-2], pid_incl_df.iloc[:, -1]
    
    return train_X, train_Y, test_X, test_Y

In [141]:
train_Y.shape, train_df.shape

((2483,), (1773, 991))

In [ ]:
output_path = "/home/ashish/Results/Datasets/Shimmer/Rowing/tsfresh/{}/"
data_type = "default"


sv = 103007
train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

train_Y = np.load(os.path.join(output_path.format(sv), "y_train_{}.npy".format(data_type)), allow_pickle=True)
test_Y = np.load(os.path.join(output_path.format(sv), "y_test_{}.npy".format(data_type)), allow_pickle=True)

train_pid = np.load(output_path.format(sv) + "TRAIN_{}_pid.npy".format(data_type), allow_pickle=True)
test_pid = np.load(output_path.format(sv) + "TEST_{}_pid.npy".format(data_type), allow_pickle=True)

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


# lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_Y)
# model_features = SelectFromModel(lsvc, prefit=True)
# train_X = model_features.transform(train_X)
# test_X = model_features.transform(test_X)


temp_pid = train_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]
train_df = pd.DataFrame(train_X)
train_df["pid"] = temp_pid
train_df["label"] = train_Y
temp_pid = test_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]
test_df = pd.DataFrame(test_X)
test_df["pid"] = temp_pid
test_df["label"] = test_Y


full_df = pd.concat([train_df, test_df], axis=0)
# full_df = shuffle(full_df)

model = LogisticRegression(C=0.01, max_iter=500)

unique_pids = list(np.unique(full_df.loc[:, "pid"]))

scores = []
accuracy_scores_mapping = {"pid": [], "score": []}

for pid in unique_pids:
    logger.info("The pid: {} ".format(pid))
    accuracy_scores_mapping["pid"].append(pid)
    train_X, train_Y, test_X, test_Y = generate_cross_val_splits(pid, full_df)
    logger.info("Training shape: {} {}".format(train_X.shape, train_Y.shape))
    logger.info("Testing shape: {} {}".format(test_X.shape, test_Y.shape))
    
    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)
    
    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False,  max_iter=1500).fit(train_X, train_Y)
    model_features = SelectFromModel(lsvc, prefit=True)
    train_X = model_features.transform(train_X)
    test_X = model_features.transform(test_X)

    model.fit(train_X, train_Y)

    pred_Y = model.predict(test_X)
    accuracy = metrics.accuracy_score(test_Y, pred_Y)
    
    scores.append(accuracy)
    accuracy_scores_mapping["score"].append(accuracy)
    logger.info("The accuracy: {} ".format(accuracy))
accuracy_scores_mapping_df = pd.DataFrame(accuracy_scores_mapping)
accuracy_scores_mapping_df.to_csv("/tmp/accuracy_cross_val_shimmer_mp_manual.csv", index=False)
logger.info("The average accuracy: {} std: {}".format(np.mean(scores), np.std(scores)))


Rowing  0.7377210770348024 std: 0.157, catch 22

In [9]:

output_path = "/home/ashish/Results/Datasets/HPE3/Rowing/tsfresh/{}/"
data_type = "default"


# for i, model in enumerate(list_clf):
# print("--------------------------- {}----------------".format(name_clf[i]))
avg_acc = []
for sv in SEED_VALUES:
    train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))
    test_X = pd.read_csv(output_path.format(sv) + "x_test_{}.csv".format(data_type))

    # train_Y = pd.read_csv(output_path.format(sv) + "y_train_{}.csv".format(data_type))
    # test_Y = pd.read_csv(output_path.format(sv) + "y_test_{}.csv".format(data_type))
    train_Y = np.load(os.path.join(output_path.format(sv), "y_train_{}.npy".format(data_type)), allow_pickle=True)
    test_Y = np.load(os.path.join(output_path.format(sv), "y_test_{}.npy".format(data_type)), allow_pickle=True)
    print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)
    

    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)

    # lsvc = LinearSVC(C=0.001, penalty="l1", dual=False,  max_iter=1500).fit(train_X, train_Y)
    # model_features = SelectFromModel(lsvc, prefit=True)
    # train_X = model_features.transform(train_X)
    # test_X = model_features.transform(test_X)
    # print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)


    # regressor = RandomForestClassifier()
    # model = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 7, cv = 3, verbose=2, 
    #                            random_state=42, n_jobs = -1)
    # svm.SVC(kernel='linear', C=0.001)  LogisticRegression(C=0.001, max_iter=150)   KNeighborsClassifier(n_neighbors=40) 
    model = LogisticRegression(C=0.01, max_iter=500) #LogisticRegression(C=0.01, max_iter=500)#LogisticRegression(C=0.001, max_iter=150)
    model.fit(train_X, train_Y)
    pred_Y = model.predict(test_X)


    accuracy = metrics.accuracy_score(test_Y, pred_Y)
    confusion_matrix = metrics.confusion_matrix(test_Y, pred_Y)
    classification_report = metrics.classification_report(test_Y, pred_Y)

    print(accuracy) #, print(confusion_matrix), print(classification_report)
    avg_acc.append(accuracy)

print(round(np.mean(avg_acc),2), np.std(avg_acc))

(1773, 5634) (760, 5634) (1773,) (760,)
0.7921052631578948
(1803, 5634) (730, 5634) (1803,) (730,)
0.736986301369863
(1773, 5634) (760, 5634) (1773,) (760,)
0.7736842105263158
0.77 0.02291086997014777


In [129]:
train_pid[:, 0]

array(['P20_R', 'P54_N', 'P38_A', ..., 'P8_R', 'P28_A', 'P38_R'],
      dtype='<U21')

In [123]:
train_X = pd.read_csv(output_path.format(sv) + "x_train_{}.csv".format(data_type))


In [130]:
temp_pid = train_pid[:, 0]
temp_pid = [i.split("_")[0] for i in temp_pid]

In [132]:
train_df = pd.DataFrame(train_X)
train_df["pid"] = temp_pid

In [133]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,337,338,339,340,341,342,343,344,345,pid
0,0.894999,0.559688,-0.690173,0.831021,0.559537,1.100548,0.065464,0.427734,0.218339,-0.570688,...,-0.446356,-0.059032,-1.072284,-0.213050,-0.026412,0.969252,1.814752,-1.871549,2.551862,P20
1,-0.651896,-1.117284,-0.441646,-0.050600,-0.779768,0.913652,0.795446,-1.974349,0.743754,-0.082648,...,-0.534210,1.103280,0.668201,-0.409789,0.273002,-0.827151,-0.962456,0.687372,-0.413999,P54
2,0.051238,0.426738,-0.469260,-1.238088,-0.658013,1.118069,0.795446,0.599311,-1.066010,-0.204658,...,1.793923,0.788091,-0.064635,-0.311419,-1.643249,0.071051,0.091768,0.687372,-0.413999,P38
3,-1.073776,0.234379,-0.093707,-0.638346,-1.266788,-1.072113,0.430455,0.599311,-0.482215,0.893433,...,-0.446356,-1.018857,0.759806,-0.114681,-0.445592,1.025390,1.808243,0.782146,-0.413999,P16
4,-0.089389,-0.711419,-0.538295,0.145316,-2.119073,-1.294051,-2.124483,0.170368,1.210790,-0.936718,...,-0.534210,-1.188977,-0.705866,-0.311419,1.470659,-1.332389,-0.962456,-0.355152,-0.413999,P30


In [ ]:
data_type = "default_noscale"
output_path = '/home/ashish/Results/Datasets/Shimmer/Rowing/Manual/103007/'
train_df = pd.read_csv(output_path + "train_{}.csv".format(data_type))
test_df = pd.read_csv(output_path + "test_{}.csv".format(data_type))


full_df = pd.concat([train_df, test_df], axis=0)

# train_df = shuffle(train_df)
# test_df = shuffle(test_df)
full_df = shuffle(full_df)

print(train_df.shape, test_df.shape, full_df.shape)


scores = []
accuracy_scores_mapping = {"pid": [], "score": []}


model = LogisticRegression(C=0.001, max_iter=150)

unique_pids = list(np.unique(full_df.iloc[:, -2]))

for pid in unique_pids:
    logger.info("The pid: {} ".format(pid))
    accuracy_scores_mapping["pid"].append(pid)
    train_X, train_Y, test_X, test_Y = generate_cross_val_splits(pid)
    logger.info("Training shape: {} {}".format(train_X.shape, train_Y.shape))
    logger.info("Testing shape: {} {}".format(test_X.shape, test_Y.shape))
    
    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)

    model.fit(train_X, train_Y)

    pred_Y = model.predict(test_X)
    accuracy = metrics.accuracy_score(test_Y, pred_Y)
    
    scores.append(accuracy)
    accuracy_scores_mapping["score"].append(accuracy)
    logger.info("The accuracy: {} ".format(accuracy))
accuracy_scores_mapping_df = pd.DataFrame(accuracy_scores_mapping)
accuracy_scores_mapping_df.to_csv("/tmp/accuracy_cross_val_shimmer_mp_manual.csv", index=False)
logger.info("The average accuracy: {} std: {}".format(np.mean(scores), np.std(scores)))